## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-06-13-05-26 +0000,nyt,Live Updates: Trump Suggests Extended U.S. Inv...,https://www.nytimes.com/live/2026/01/06/world/...
1,2026-01-06-12-58-36 +0000,nypost,Death row inmate reveals heartbreaking email h...,https://nypost.com/2026/01/06/us-news/death-ro...
2,2026-01-06-12-56-45 +0000,wapo,NRA sues its own charity in growing schism ove...,https://www.washingtonpost.com/nation/2026/01/...
3,2026-01-06-12-55-32 +0000,nyt,Can VenezuelaNew Leader Keep the U.S. Happy an...,https://www.nytimes.com/2026/01/06/world/can-v...
4,2026-01-06-12-53-02 +0000,nyt,Swiss Bar Hit by Deadly Fire Was Not Inspected...,https://www.nytimes.com/2026/01/06/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2362/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,venezuela,58
263,maduro,53
2,trump,52
333,capture,18
106,new,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
222,2026-01-05-21-27-39 +0000,cbc,"Venezuela's collapse, Trump’s military buildup...",https://www.cbc.ca/news/world/trump-venezuela-...,194
253,2026-01-05-19-32-24 +0000,nyt,"Who Is Delcy Rodríguez, Venezuela’s New Interi...",https://www.nytimes.com/2026/01/05/world/ameri...,170
243,2026-01-05-20-11-39 +0000,nypost,Elon Musk’s Starlink offers free internet serv...,https://nypost.com/2026/01/05/business/elon-mu...,154
194,2026-01-05-22-28-00 +0000,wsj,Venezuela’s New Leader Rushes to Assert Contro...,https://www.wsj.com/world/americas/venezuela-d...,150
258,2026-01-05-19-05-33 +0000,nyt,Trump’s Goals for Reviving Venezuela’s Oil Ind...,https://www.nytimes.com/2026/01/05/business/en...,149


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
222,194,2026-01-05-21-27-39 +0000,cbc,"Venezuela's collapse, Trump’s military buildup...",https://www.cbc.ca/news/world/trump-venezuela-...
52,92,2026-01-06-10-33-43 +0000,nypost,"Tim Walz’s firebrand daughter, Hope, admits th...",https://nypost.com/2026/01/06/us-news/tim-walz...
136,63,2026-01-06-02-24-00 +0000,wsj,In the wake of the Trump administration cuttin...,https://www.wsj.com/politics/policy/trump-admi...
33,57,2026-01-06-11-53-09 +0000,nypost,Venezuelan opposition leader declares she woul...,https://nypost.com/2026/01/06/world-news/venez...
311,46,2026-01-05-15-25-42 +0000,nypost,Paris court finds 10 people guilty of cyberbul...,https://nypost.com/2026/01/05/world-news/paris...
316,41,2026-01-05-15-07-08 +0000,nypost,"Chevron, energy stocks soar after US capture o...",https://nypost.com/2026/01/05/business/chevron...
170,40,2026-01-05-23-50-43 +0000,nypost,Iran’s Ayatollah Ali Khamenei plans to flee to...,https://nypost.com/2026/01/05/world-news/irans...
54,32,2026-01-06-10-30-23 +0000,bbc,Nvidia unveils self-driving car tech as it see...,https://www.bbc.com/news/articles/c0jv1vd571wo...
217,32,2026-01-05-21-36-00 +0000,wsj,U.S. companies will benefit under a new global...,https://www.wsj.com/politics/policy/oecd-corpo...
30,31,2026-01-06-11-57-13 +0000,nyt,‘No One Is Ready for This’: How a Deadly Bar F...,https://www.nytimes.com/2026/01/04/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
